In [25]:
import json
import numpy as np

In [11]:
musique_dpr = json.load(open('/research/nfs_su_809/workspace/shu.251/HippoRAG/output/ircot/ircot_results_musique_dpr_only_R_facebook_contriever_L_facebook_contriever_demo_1_E_gpt-3_5-turbo_doc_ensemble_0.9_step_1_top_8_ppr_damping_0.1_sim_thresh_0.8.json','r'))

In [14]:
wiki2_dpr = json.load(open('/research/nfs_su_809/workspace/shu.251/HippoRAG/output/ircot/ircot_results_2wikimultihopqa_dpr_only_R_facebook_contriever_L_facebook_contriever_demo_1_E_gpt-3_5-turbo_doc_ensemble_0.9_step_1_top_8_ppr_damping_0.1_sim_thresh_0.8.json','r'))

In [15]:
wiki2_hip = json.load(open('/research/nfs_su_809/workspace/shu.251/HippoRAG/output/ircot/ircot_results_2wikimultihopqa_hipporag_R_facebook_contriever_L_facebook_contriever_demo_1_E_gpt-3_5-turbo_no_ensemble_step_1_top_8_ppr_damping_0.1_sim_thresh_0.8.json','r'))

In [16]:
musique_hip = json.load(open('/research/nfs_su_809/workspace/shu.251/HippoRAG/output/ircot/ircot_results_musique_hipporag_R_facebook_contriever_L_facebook_contriever_demo_1_E_gpt-3_5-turbo_no_ensemble_step_1_top_8_ppr_damping_0.1_sim_thresh_0.8.json','r'))

In [68]:
def get_second_supporting_doc_metric(docs,musique=True):

    supp_doc_rank = []
    
    for example in docs:
        
        if musique:
            supp_docs = set([d['title'] + '\n' + d['paragraph_text'] for d in example['supporting_docs']])
        else:
            gold_passages = [item for item in example['supporting_facts']]
            supp_docs = set([item[0] for item in gold_passages])

        ranks = []
        
        retrieved_docs = example['retrieved']
        
        if not(musique):
            retrieved_docs = [passage.split('\n')[0].strip() for passage in retrieved_docs]

        for i, ret in enumerate(retrieved_docs):
            if ret in supp_docs:
                supp_docs.remove(ret)
                ranks.append(i)

            if len(ranks) == 2:
                break
                
        if len(ranks) == 1:
            ranks.append(9)
        # elif len(ranks) == 0:
        #     ranks.append(10)
        #     ranks.append(10)
        if len(ranks) == 2:    
            supp_doc_rank.append(ranks[1] - ranks[0])

    return np.array(supp_doc_rank)

In [69]:
get_second_supporting_doc_metric(musique_dpr).mean(axis=0)

5.683807439824945

In [70]:
get_second_supporting_doc_metric(wiki2_dpr, musique=False).mean(axis=0)

5.456544502617801

In [71]:
get_second_supporting_doc_metric(musique_hip).mean(axis=0)

5.6404682274247495

In [72]:
get_second_supporting_doc_metric(wiki2_hip, musique=False).mean(axis=0)

2.1012145748987856

In [104]:
def get_recall(docs):
    recalls = []
    all_recalls = []
    
    for doc in docs:
        recalls.append(doc['recall'])
        all_recalls.append(doc['all_recall'])

    display(pd.DataFrame(recalls).agg(['mean']))
    # display(pd.DataFrame(all_recalls).agg(['mean']))

In [105]:
get_recall(musique_dpr)

,1,2,5,10,20
mean,0.241577,0.348246,0.46625,0.525167,0.525167


In [106]:
for type in np.unique([d['type'] for d in wiki2_dpr]):
    print(type)
    get_recall([d for d in wiki2_dpr if d['type'] == type])
    get_recall([d for d in wiki2_hip if d['type'] == type])

bridge_comparison


,1,2,5,10,20
mean,0.179787,0.273404,0.358511,0.390426,0.390426


,1,2,5,10,20
mean,0.247872,0.487234,0.898936,0.954255,0.954255


comparison


,1,2,5,10,20
mean,0.428279,0.67623,0.838115,0.889344,0.889344


...

,1,2,5,10,20
mean,0.360775,0.453995,0.539952,0.571429,0.571429


,1,2,5,10,20
mean,0.456416,0.745763,0.864407,0.903148,0.903148


inference


,1,2,5,10,20
mean,0.333333,0.453704,0.587963,0.652778,0.652778


,1,2,5,10,20
mean,0.435185,0.601852,0.768519,0.856481,0.856481
